In [74]:
import json
import pandas as pd

# Load the JSON data from the file
with open('exercises.json', 'r') as f:
    data = json.load(f)

name, force, level, equipment, primary_muscles, secondary_muscles, category = [], [], [], [], [], [], []
equipment_filter = ['barbell', 'dumbbell', 'machine', "e-z curl bar", "cable", "body only"]

# Iterate over each muscle group and each exercise within that group
for muscle_group in data:
    for exercise in data[muscle_group]:
        if exercise.get('equipment') in equipment_filter:
            name.append(exercise['name'])
            level.append(exercise.get('level', 'N/A'))  # Assuming 'level' corresponds to 'difficulty'
            equipment.append(exercise.get('equipment', 'N/A'))
            primary_muscles.append(exercise.get('primaryMuscles'))
            secondary_muscles.append(exercise.get('secondaryMuscles'))
            category.append(exercise.get('category'))

# Create a DataFrame
df = pd.DataFrame({'name': name, 'level': level, 'equipment': equipment, 'primary_muscles': primary_muscles, 'secondary_muscles': secondary_muscles, 'category': category})

strength_df = df[df['category'] == 'strength'].copy()

# Display the first few rows of the DataFrame
strength_df.head(25)

,name,level,equipment,primary_muscles,secondary_muscles,category
0,3/4 Sit-Up,beginner,body only,[abdominals],[],strength
2,Ab Crunch Machine,intermediate,machine,[abdominals],[],strength
4,Air Bike,beginner,body only,[abdominals],[],strength
6,Alternate Hammer Curl,beginner,dumbbell,[biceps],[forearms],strength
7,Alternate Heel Touchers,beginner,body only,[abdominals],[],strength
8,Alternate Incline Dumbbell Curl,beginner,dumbbell,[biceps],[forearms],strength
10,Alternating Cable Shoulder Press,beginner,cable,[shoulders],[triceps],strength
11,Alternating Deltoid Raise,beginner,dumbbell,[shoulders],[],strength
14,Anti-Gravity Press,beginner,barbell,[shoulders],"[middle back, traps, triceps]",strength
16,Arnold Dumbbell Press,intermediate,dumbbell,[shoulders],[triceps],strength


In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Combine features into a single string
# Combine features into a single string
strength_df['features'] = strength_df.apply(lambda x: ' '.join(x['primary_muscles'] + x['secondary_muscles']), axis=1)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the features
tfidf_matrix = vectorizer.fit_transform(strength_df['features'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Define workout categories
push_muscles = ['chest', 'shoulders', 'triceps']
pull_muscles = ['back', 'biceps']
legs_muscles = ['quads', 'hamstrings', 'glutes']


In [76]:
# Function to get recommendations based on similarity
def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the exercise
    idx = strength_df[strength_df['name'] == name].index[0]

    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the exercises based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar exercises
    sim_scores = sim_scores[1:6]

    # Get the exercise indices
    exercise_indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar exercises
    return strength_df['name'].iloc[exercise_indices]



In [83]:
print("Recommendations for 'Pullups':")
print(get_recommendations('Pullups'))

Recommendations for 'Pullups':
52     Bent Over Dumbbell Rear Delt Raise With Head O...
82                               Cable Internal Rotation
88                                   Cable Rear Delt Fly
169                    Dumbbell Lying Rear Lateral Raise
197                                    External Rotation
Name: name, dtype: object


In [86]:
df['features'] = df.apply(lambda x: ' '.join(x['primary_muscles']), axis=1)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the features
tfidf_matrix = vectorizer.fit_transform(df['features'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the exercise
    idx = df[df['name'] == name].index[0]

    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the exercises based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar exercises
    sim_scores = sim_scores[1:6]

    # Get the exercise indices
    exercise_indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar exercises
    return df['name'].iloc[exercise_indices]

# Example usage
print("Recommendations for 'Bench Press':")
print(get_recommendations('Pullups'))

AttributeError: 'list' object has no attribute 'lower'